<a href="https://colab.research.google.com/github/HaipingWu2001/suanming_model_fine_tuning/blob/main/Fine_tuning_magic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 第一步：安装依赖

In [2]:
%%capture
# 这是一个 Jupyter Notebook 的魔法命令，用于隐藏命令的输出，让笔记本界面更整洁。

# 安装 unsloth 包。unsloth 是一个用于微调大型语言模型（LLM）的工具，可以让模型运行更快、占用更少内存。
!pip install unsloth

# 卸载当前已安装的 unsloth 包（如果已安装），然后从 GitHub 的源代码安装最新版本。
# 这样可以确保我们使用的是最新功能和修复。
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

# 安装 bitsandbytes 和 unsloth_zoo 包。
# bitsandbytes 是一个用于量化和优化模型的库，可以帮助减少模型占用的内存。
# unsloth_zoo 可能包含了一些预训练模型或其他工具，方便我们使用。
!pip install bitsandbytes unsloth_zoo

## 第二步：加载预训练模型

In [3]:
from unsloth import FastLanguageModel  # 导入FastLanguageModel类，用来加载和使用模型
import torch  # 导入torch工具，用于处理模型的数学运算

max_seq_length = 2048  # 设置模型处理文本的最大长度，相当于给模型设置一个“最大容量”
dtype = None  # 设置数据类型，让模型自动选择最适合的精度
load_in_4bit = True  # 使用4位量化来节省内存，就像把大箱子压缩成小箱子

# 加载预训练模型，并获取tokenizer工具
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B",  # 指定要加载的模型名称
    max_seq_length=max_seq_length,  # 使用前面设置的最大长度
    dtype=dtype,  # 使用前面设置的数据类型
    load_in_4bit=load_in_4bit,  # 使用4位量化
    # token="hf_...",  # 如果需要访问授权模型，可以在这里填入密钥
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.17: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

## 第三步：微调前测试

In [4]:
prompt_style = """以下是描述任务的指令，以及提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

### 指令：
你是一位精通卜卦、星象和运势预测的算命大师。
请回答以下算命问题。

### 问题：
{}

### 回答：
<think>{}"""
# 定义提示风格的字符串模板，用于格式化问题

question = "1992年闰四月初九巳时生人，女，想了解健康运势"
# 定义具体的算命问题

In [5]:
FastLanguageModel.for_inference(model)
# 准备模型以进行推理

inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")
# 使用 tokenizer 对格式化后的问题进行编码，并移动到 GPU

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
# 使用模型生成回答

response = tokenizer.batch_decode(outputs)
# 解码模型生成的输出为可读文本

print(response[0])
# 打印生成的回答部分

<｜begin▁of▁sentence｜>以下是描述任务的指令，以及提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

### 指令：
你是一位精通卜卦、星象和运势预测的算命大师。
请回答以下算命问题。

### 问题：
1992年闰四月初九巳时生人，女，想了解健康运势

### 回答：
<think>
嗯，用户是一位1992年闰四月初九巳时出生的女性，想了解自己的健康运势。首先，我需要分析她的出生日期和时间，确定她的生肖和运势。1992年属于中国的农历年，闰四月初九对应的是牛年，因为牛年通常从春节到夏至为周期。

接下来，巳时是中国传统时辰之一，属于地支之一。巳时通常与“蛇”相关联，代表灵巧、智慧和变化。因此，她的生肖应该是蛇。

蛇女在命理学中通常被认为是聪明、有才华、善于交际的人。她的健康方面可能需要关注一些方面，比如可能存在一些隐形的健康问题，需要注意心脏健康和免疫系统的调理。同时，作为蛇女，她可能在情感上比较独立，需要平衡好自己的情感需求和事业发展。

在健康方面，建议她多注意身体的调养，适当进行运动和饮食控制，避免过度劳累。情感上，她可以通过与亲友和朋友的交流来获得支持，保持心理健康。事业上，她可能会经历一些变化，需要保持积极的态度和灵活的应对能力。

总的来说，蛇女的健康运势需要她关注自己的身体状态，保持良好的生活习惯，同时注意情感和事业的平衡。通过这些方面的调养，她可以保持良好的健康状况，实现个人价值和生活目标。
</think>

根据您的出生日期和时间，1992年闰四月初九巳时，您的生肖是蛇（蛇女）。蛇女在命理学中通常具有智慧、灵动和敏锐的特质。

在健康方面，您需要特别注意以下几点：
1. **心脏健康**：蛇女可能存在一些潜在的心脏问题，建议定期进行心电图检查，保持心脏健康。
2. **免疫调理**：如果您有过敏症或免疫系统问题，建议咨询医生进行针对性调理。
3. **身体调养**：保持规律的作息，适当运动，如瑜伽或散步，增强体质。

在情感方面，蛇女通常比较独立，但也需要学会与他人建立深厚的关系。建议您多与亲友交流，保持良好的家庭关系和社会关系。

在事业发展方面，蛇女通常有很强的适应能力和创新思维，建议您多参加行业交流活动，拓展人脉资源，寻找发展机会。

## 第四步：加载数据集

In [7]:
# 定义一个用于格式化提示的多行字符串模板
train_prompt_style = """以下是描述任务的指令，以及提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

### 指令：
你是一位精通八字算命、 紫微斗数、 风水、易经卦象、塔罗牌占卜、星象、面相手相和运势预测等方面的算命大师。
请回答以下算命问题。

### 问题：
{}

### 回答：
<思考>
{}
</思考>
{}"""

In [8]:
# 定义结束标记（EOS_TOKEN），用于指示文本的结束
EOS_TOKEN = tokenizer.eos_token  # 必须添加结束标记

# 导入数据集加载函数
from datasets import load_dataset
# 加载指定的数据集，选择中文语言和训练集的前200条记录
dataset = load_dataset("Conard/fortune-telling", 'default', split = "train[0:200]", trust_remote_code=True)
# 打印数据集的列名，查看数据集中有哪些字段
print(dataset.column_names)

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

all_details.json:   0%|          | 0.00/693k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/207 [00:00<?, ? examples/s]

['Question', 'Response', 'Complex_CoT']


In [9]:
# 定义一个函数，用于格式化数据集中的每条记录
def formatting_prompts_func(examples):
    # 从数据集中提取问题、复杂思考过程和回答
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []  # 用于存储格式化后的文本
    # 遍历每个问题、思考过程和回答，进行格式化
    for input, cot, output in zip(inputs, cots, outputs):
        # 使用字符串模板插入数据，并加上结束标记
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)  # 将格式化后的文本添加到列表中
    return {
        "text": texts,  # 返回包含所有格式化文本的字典
    }

dataset = dataset.map(formatting_prompts_func, batched = True)
dataset["text"][0]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

'以下是描述任务的指令，以及提供进一步上下文的输入。\n请写出一个适当完成请求的回答。\n在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。\n\n### 指令：\n你是一位精通八字算命、 紫微斗数、 风水、易经卦象、塔罗牌占卜、星象、面相手相和运势预测等方面的算命大师。\n请回答以下算命问题。\n\n### 问题：\n新房装修,大门对着电梯好不好?要如何化解?\n\n### 回答：\n<思考>\n好的，用户问的是新房装修时大门对着电梯好不好，以及如何化解。首先，我需要回忆一下风水学中关于大门和电梯的相关知识。电梯在风水中属于动气比较强的地方，因为电梯频繁开合，会带来不稳定的气流，也就是所谓的“煞气”。大门是住宅的纳气口，如果正对电梯，可能会让这些不稳定的气流直接冲进家里，影响居住者的健康和财运。\n\n接下来，我需要确认用户的具体情况。比如，大门和电梯的距离有多远？是否正对还是稍微偏一点？不过用户没有提供这些细节，所以只能给出一般性的建议。化解的方法通常有几种：屏风或玄关、门帘、五帝钱、植物、八卦镜等。需要逐一解释这些方法的原理和使用方式，同时提醒用户要根据实际情况选择，必要时咨询专业风水师。\n\n另外，还要注意语气要亲切，避免使用过于专业的术语，让用户容易理解。同时，要强调这些是传统方法，效果因人而异，保持客观中立。最后，可以建议用户如果情况复杂，最好请专业人士实地查看，这样更稳妥。\n\n</思考>\n根据传统风水学的观点，大门正对电梯易形成"开口煞"，电梯频繁升降会扰乱家宅气场。建议化解方案：\n\n1. 玄关阻隔法\n在入门处设置L型屏风或文化砖玄关墙，高度以1.8米为宜，既保持采光又形成缓冲带\n\n2. 五行通关法\n门槛石下埋设五帝钱+白玉葫芦，建议选丙申年铸造的真品古币，配合门楣悬挂九宫八卦镜\n\n3. 光影化解术\n安装磨砂玻璃内推门，门框镶嵌黄铜门槛，每日辰时用海盐净化门廊区域\n\n4. 现代科技方案\n入户区安装智能感应灯带，设置循环播放的流水声效，运用声光电技术平衡磁场\n\n需注意电梯井方位与家主命卦的关系，建议提供具体户型平面图进行吉凶方位测算。当代建筑中可采用半透明艺术隔断结合空气净化系统，既符合科学原理又兼顾传统智慧。<｜end▁of▁sentence｜>'

## 第五步：执行微调

In [10]:
FastLanguageModel.for_training(model) #将模型切换至训练模式用于微调

model = FastLanguageModel.get_peft_model(
    model,  # 传入已经加载好的预训练模型
    r = 16,  # 设置 LoRA 的秩，决定添加的可训练参数数量
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",  # 指定模型中需要微调的关键模块
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,  # 设置 LoRA 的超参数，影响可训练参数的训练方式
    lora_dropout = 0,  # 设置防止过拟合的参数，这里设置为 0 表示不丢弃任何参数
    bias = "none",    # 设置是否添加偏置项，这里设置为 "none" 表示不添加
    use_gradient_checkpointing = "unsloth",  # 使用优化技术节省显存并支持更大的批量大小
    random_state = 114511,  # 设置随机种子，确保每次运行代码时模型的初始化方式相同
    use_rslora = False,  # 设置是否使用 Rank Stabilized LoRA 技术，这里设置为 False 表示不使用
    loftq_config = None,  # 设置是否使用 LoftQ 技术，这里设置为 None 表示不使用
)

Unsloth 2025.3.17 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [11]:
from trl import SFTTrainer  # 导入 SFTTrainer，用于监督式微调
from transformers import TrainingArguments  # 导入 TrainingArguments，用于设置训练参数
from unsloth import is_bfloat16_supported  # 导入函数，检查是否支持 bfloat16 数据格式

# 设置训练超参数
trainer = SFTTrainer(  # 创建一个 SFTTrainer 实例
    model=model,  # 传入要微调的模型
    tokenizer=tokenizer,  # 传入 tokenizer，用于处理文本数据
    train_dataset=dataset,  # 传入训练数据集
    dataset_text_field="text",  # 指定数据集中文本字段的名称
    max_seq_length=max_seq_length,  # 设置最大序列长度
    dataset_num_proc=2,  # 设置数据处理的并行进程数
    packing=False,  # 是否启用打包功能（这里设置为 False，打包可以让训练更快，但可能影响效果）
    args=TrainingArguments(  # 定义训练参数
        per_device_train_batch_size=2,  # 每个设备（如 GPU）上的批量大小
        gradient_accumulation_steps=4,  # 梯度累积步数，用于模拟大批次训练
        warmup_steps=5,  # 预热步数，训练开始时学习率逐渐增加的步数
        max_steps=75,  # 最大训练步数
        learning_rate=2e-4,  # 学习率，模型学习新知识的速度
        fp16=not is_bfloat16_supported(),  # 是否使用 fp16 格式加速训练（如果环境不支持 bfloat16）
        bf16=is_bfloat16_supported(),  # 是否使用 bfloat16 格式加速训练（如果环境支持）
        logging_steps=1,  # 每隔多少步记录一次训练日志
        optim="adamw_8bit",  # 使用的优化器，用于调整模型参数
        weight_decay=0.01,  # 权重衰减，防止模型过拟合
        lr_scheduler_type="linear",  # 学习率调度器类型，控制学习率的变化方式
        seed=114511,  # 随机种子，确保训练结果可复现
        output_dir="outputs",  # 训练结果保存的目录
        report_to="none",  # 是否将训练结果报告到外部工具（如 WandB），这里设置为不报告
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 200 | Num Epochs = 3 | Total steps = 75
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.492900
2,2.342100
3,2.116400
4,2.184100
5,2.232500
6,2.129700
7,2.198100
8,1.979800
9,2.079200
10,1.977300


## 第七步：微调后测试

In [ ]:
print(question) # 打印前面的问题

In [ ]:
# 将模型切换到推理模式，准备回答问题
FastLanguageModel.for_inference(model)

# 将问题转换成模型能理解的格式，并发送到 GPU 上
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# 让模型根据问题生成回答，最多生成 4000 个新词
outputs = model.generate(
    input_ids=inputs.input_ids,  # 输入的数字序列
    attention_mask=inputs.attention_mask,  # 注意力遮罩，帮助模型理解哪些部分重要
    max_new_tokens=4000,  # 最多生成 4000 个新词
    use_cache=True,  # 使用缓存加速生成
)

# 将生成的回答从数字转换回文字
response = tokenizer.batch_decode(outputs)

# 打印回答
print(response[0])

## 第八步：将微调后的模型保存为 GGUF 格式



In [ ]:
# 导入 Google Colab 的 userdata 模块，用于访问用户数据
from google.colab import userdata

# 从 Google Colab 用户数据中获取 Hugging Face 的 API 令牌
HUGGINGFACE_TOKEN = userdata.get('HUGGINGFACE_TOKEN')

# 将模型保存为 8 位量化格式（Q8_0）
# 这种格式文件小且运行快，适合部署到资源受限的设备
if True: model.save_pretrained_gguf("model", tokenizer,)

# 将模型保存为 16 位量化格式（f16）
# 16 位量化精度更高，但文件稍大
if False: model.save_pretrained_gguf("model_f16", tokenizer, quantization_method = "f16")

# 将模型保存为 4 位量化格式（q4_k_m）
# 4 位量化文件最小，但精度可能稍低
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")